In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import json

<h5>Cleaning up Yahoo data</h5>

In [2]:
def clean_yahoo_df(path):
    ticker = path.split('/')[-1].split('.')[0]
    df = pd.read_csv(path)
    
    df.drop(['High','Low','Open','Close'], axis=1, inplace=True)
    df['Adj Close'] = df['Adj Close'].round(2)
    #df['Volume'] = df['Volume'].astype('int32')
    df.rename(columns={'Volume': f'{ticker}_volume', 'Adj Close': ticker}, inplace=True)
    df.set_index('Date', inplace=True)
    
    return df

In [3]:
main_df = clean_yahoo_df('final_dfs/spy.csv')
main_df.head()

,spy_volume,spy
Date,,
1993-01-29,1003200.0,26.30
1993-02-01,480500.0,26.49
1993-02-02,201300.0,26.54
1993-02-03,529400.0,26.82
1993-02-04,531500.0,26.94


Below are all tickers pulled from Yahoo and saved locally. I will need to find a way to automate the refresh daily but for now will use the data I have to optimize the model.

Also note, for the individual sector ETFs, I will use Vanguard ETFs instead of SPDR even if SPY is an SPDR ETFs. Even though most of the SPDR ETFs are larger and more liquid I find the Vanguard ETFs more accurately reflects their respective idustries as the portfolios include small and mid caps. SPDR ETFs are heavily concentrated in stocks already included in SPY.

In [4]:
# note: leaving out USO becuase we have /CL

etfs = ['qqq', 'iwm']

sector_etf_path = 'sector etfs - Vanguard'
sector_etf = ['vaw', 'vcr', 'vdc', 'vde', 'vfh', 'vgt', 'vht', 'vis', 'vnq', 'vox', 'vpu']

futures_path = 'futures'
futures = ['cl=f', 'gc=f']

bonds_etf_path = 'bonds etfs'
bonds = ['ief', 'tlt']

In [5]:
for ticker in etfs:
    temp_df = clean_yahoo_df(f'final_dfs/{ticker}.csv')
    main_df = main_df.join(temp_df, how='left')

for ticker in sector_etf:
    temp_df = clean_yahoo_df(f'final_dfs/{sector_etf_path}/{ticker}.csv')
    main_df = main_df.join(temp_df, how='left')

for ticker in futures:
    temp_df = clean_yahoo_df(f'final_dfs/{futures_path}/{ticker}.csv')
    main_df = main_df.join(temp_df, how='left')
    
for ticker in bonds:
    temp_df = clean_yahoo_df(f'final_dfs/{bonds_etf_path}/{ticker}.csv')
    main_df = main_df.join(temp_df, how='left')

In [6]:
main_df.head()

,spy_volume,spy,qqq_volume,qqq,iwm_volume,iwm,vaw_volume,vaw,vcr_volume,vcr,...,vpu_volume,vpu,cl=f_volume,cl=f,gc=f_volume,gc=f,ief_volume,ief,tlt_volume,tlt
Date,,,,,,,,,,,,,,,,,,,,,
1993-01-29,1003200.0,26.30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1993-02-01,480500.0,26.49,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1993-02-02,201300.0,26.54,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1993-02-03,529400.0,26.82,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1993-02-04,531500.0,26.94,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Other charts (Treasure bond yields, Libor rate, Mortgage rate, M1 & M2 money supply, VIX, DIX)

In [7]:
# maybe include VVIX in the future?
# Notes:
# 30 year fixed mortgage rate is updated monthly (need may numbers)
# money supply data is updated weekly


yields_path = 'yields'
treasury_yields = ['10-year-treasury-bond-rate-yield-chart', '30-year-fixed-mortgage-rate-chart',
                   '30-year-treasury-bond-rate-yield-chart', 'fed-funds-rate-historical-chart']
libor_yields = ['historical-libor-rates-chart']

money_supply_path = 'money supply'
money_supply = ['M1', 'M2']

currency_path = 'currency'
currency = ['dxy']

vix = '^vix'
dix = 'DIX'

In [22]:
# # treasury/mortgage rate
# for i in treasury_yields:
#     bonds_df = pd.read_csv(f'final_dfs/{yields_path}/{i}.csv', skiprows=15)
#     bonds_df[' value'] = bonds_df[' value'] / 100
#     bonds_df.rename(columns={' value': f'{i}'.replace('-chart', '')}, inplace=True)
#     bonds_df.set_index('date', inplace=True)
#     main_df = main_df.join(bonds_df, how='left')
    
# libor rate
libor_df = pd.read_csv(f'final_dfs/{yields_path}/{libor_yields[0]}.csv', skiprows=15)
libor_df.rename(columns={'1M': '1M_libor_rate',
                         '3M': '3M_libor_rate',
                         '6M': '6M_libor_rate',
                         '12M': '12M_libor_rate'},
               inplace=True)
#libor_df.set_index('date', inplace=True)
#libor_df = libor_df / 100
# main_df = main_df.join(libor_df, how='left')
libor_df.head()

# money supply
# for i in money_supply:
#     money_supply_df = pd.read_csv(f'final_dfs/{money_supply_path}/{i}.csv')
#     money_supply_df.rename(columns={'M1': 'M1_money', 'M2': 'M2_money'}, inplace=True)
#     money_supply_df.set_index('DATE', inplace=True)
#     money_supply_df = money_supply_df * 1000000000
#     main_df = main_df.join(money_supply_df, how='left')

# # currency (check date format)
# currency_df = pd.read_csv(f'final_dfs/{currency_path}/{currency[0]}.csv')
# currency_df.drop([' Open',' High',' Low'], axis=1, inplace=True)
# currency_df.rename(columns={' Close': currency[0]}, inplace=True)
# currency_df.set_index('Date', inplace=True)
# main_df = main_df.join(currency_df, how='left')

# # vix
# vix_df = clean_yahoo_df(f'final_dfs/{vix}.csv')
# vix_df.drop('^vix_volume', axis=1, inplace=True)
# main_df = main_df.join(vix_df, how='left')

# # dix
# dix_df = pd.read_csv(f'final_dfs/{dix}.csv')
# dix_df.drop('price', axis=1, inplace=True)
# dix_df.set_index('date', inplace=True)
# main_df = main_df.join(dix_df, how='left')

,date,1M_libor_rate,3M_libor_rate,6M_libor_rate,12M_libor_rate
0,01/01/1986,8.25,8.25,8.25,8.50
1,02/01/1986,7.94,7.88,7.88,7.94
2,03/01/1986,7.50,7.44,7.44,7.44
3,04/01/1986,6.94,6.88,6.88,6.94
4,05/01/1986,7.13,7.13,7.19,7.38


In [19]:
main_df.iloc[-30:,35:]

,tlt,10-year-treasury-bond-rate-yield,30-year-fixed-mortgage-rate,30-year-treasury-bond-rate-yield,fed-funds-rate-historical,1M_libor_rate,3M_libor_rate,6M_libor_rate,12M_libor_rate,M1_money,M2_money,dxy,^vix,dix,gex
Date,,,,,,,,,,,,,,,
2020-03-20,158.99,0.00920,NaN,0.0155,0.0015,NaN,NaN,NaN,NaN,NaN,NaN,NaN,66.04,0.411404,-6.417786e+08
2020-03-23,165.55,0.00760,NaN,0.0133,0.0015,NaN,NaN,NaN,NaN,4.323700e+12,1.627040e+13,NaN,61.59,0.366571,-6.780356e+07
2020-03-24,162.46,0.00840,NaN,0.0139,0.0012,NaN,NaN,NaN,NaN,NaN,NaN,NaN,61.67,0.402208,6.002200e+08
2020-03-25,162.10,0.00880,NaN,0.0145,0.0010,NaN,NaN,NaN,NaN,NaN,NaN,NaN,63.95,0.461185,-1.576467e+08
2020-03-26,162.89,0.00830,NaN,0.0142,0.0010,NaN,NaN,NaN,NaN,NaN,NaN,NaN,61.00,0.480941,1.106730e+09
2020-03-27,167.24,0.00720,NaN,0.0129,0.0010,NaN,NaN,NaN,NaN,NaN,NaN,NaN,65.54,0.482826,-2.893855e+08
2020-03-30,165.86,0.00700,NaN,0.0131,0.0009,NaN,NaN,NaN,NaN,4.525800e+12,1.666600e+13,NaN,57.08,0.472740,9.862025e+07
2020-03-31,164.52,0.00700,NaN,0.0135,0.0008,NaN,NaN,NaN,NaN,NaN,NaN,NaN,53.54,0.497417,-1.482108e+08
2020-04-01,166.82,0.00620,0.0323,0.0127,0.0006,NaN,NaN,NaN,NaN,NaN,NaN,NaN,57.06,0.511401,-7.728500e+08


In [ ]:
# change datetime format for libor data